<a href="https://colab.research.google.com/github/APNPAPNU/Destiny2-API-Info-wiki/blob/master/og_war_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install required extensions
!pip install xvfbwrapper
!pip install pandas
!pip install BeautifulSoup
!pip install pyvirtualdisplay
!pip install kaggle
!pip install folium
!pip install pycountry-convert


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for xvfbwrapper: filename=xvfbwrapper-0.2.9-py3-none-any.whl size=5023 sha256=2e911ac046d62354836dc07804724db2b866e43c0f7ecdab2aa7a157a74a935d
  Stored in directory: /root/.cache/pip/wheels/e1/da/b4/57ac130c024104997ae76f389fe0e7e43922ec3cfdffaf1b1e
Successfully built xvfbwrapper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─

You need to create a (https://www.kaggle.com) account to acess the updated data frames. 

**once you have created your account**

**Go to your Kaggle account settings page**

**Scroll down to the "API" section**

 **Click "Create New API Token"**

**Save the kaggle.json file that's downloaded to your local machine**

 ***(remember where this file is saved as you will need it for the next step ***

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jonathanpettit","key":"2c38bd03e9f507ebcf8d85186bd209b2"}'}

In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi

# Instantiate the Kaggle API client
api = KaggleApi()
api.authenticate()

# Download the dataset
api.dataset_download_files('jonathanpettit/military-data')

# Extract the dataset and read it into a Pandas DataFrame



In [ ]:
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi

# Instantiate the Kaggle API client
api = KaggleApi()
api.authenticate()

# Download the dataset
api.dataset_download_files('jonathanpettit/military-data')

# Extract the dataset and read it into a Pandas DataFrame
!unzip military-data.zip
df = pd.read_csv('GLOBAL_FIREPOWER.csv')

# Save the DataFrame as a CSV file
df.to_csv('military_data.csv', index=False)


Archive:  military-data.zip
replace GLOBAL_FIREPOWER.CSV? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


FileNotFoundError: ignored

In [ ]:
#@title Find Nuclear Powers
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Fetch FAS data
url_fas = 'https://fas.org/issues/nuclear-weapons/status-world-nuclear-forces/'
page_fas = requests.get(url_fas)
soup_fas = BeautifulSoup(page_fas.content, 'html.parser')

# Find FAS table and read data into a pandas DataFrame
table_fas = soup_fas.find_all('table')[0]
data_fas = pd.read_html(str(table_fas))[0]

# Print column names to inspect
print(data_fas.columns)

# Set column names and select relevant columns
data_fas.columns = ['Country', 'Column 1', 'Column 2', 'Deployed Warheads', 'Total Inventory', 'Estimated Warheads']
data_fas = data_fas[['Country', 'Deployed Warheads', 'Total Inventory', 'Estimated Warheads']]

# Write data to CSV file
data_fas.to_csv('Nuclear_powers.csv', index=False)

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')


In [ ]:
#@title Find countries, capitals and regions.
import csv
import requests

response = requests.get('https://restcountries.com/v2/all')
countries = response.json()

with open('countries.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['name', 'capital', 'region', 'subregion'])
    for country in countries:
        capital = country.get('capital', 'N/A')
        writer.writerow([country['name'], capital, country['region'], country['subregion']])


In [ ]:
#@title Shift or ctrl select mutiple countries to add mutiple countries at once, other wise individually clicking will only add 1 country 
import csv
import requests
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Pull country data and write to CSV file
response = requests.get('https://restcountries.com/v2/all')
countries = response.json()
with open('countries.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['name', 'capital', 'region', 'subregion'])
    for country in countries:
        capital = country.get('capital', 'N/A')
        writer.writerow([country['name'], capital, country['region'], country['subregion']])

# Read the CSV file for attackers into a pandas dataframe
df1 = pd.read_csv('countries.csv')

# Extract the names of the countries into a list
country_names1 = df1['name'].tolist()

# Create the dropdown widget for attackers
attacker_selection = widgets.SelectMultiple(
    options=country_names1,
    value=[],
    description='Attackers:',
    disabled=False
)

# Display the attackers dropdown widget
display(attacker_selection)

# Create the button to save attackers
attacker_save_button = widgets.Button(
    description='Save Attackers',
    disabled=False,
    button_style='',
    tooltip='Save selected attackers'
)

# Define the function to save attackers
def save_attackers(_):
    with open('attackers.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['name'])
        for attacker in attacker_selection.value:
            writer.writerow([attacker])

# Attach the save function to the attackers button
attacker_save_button.on_click(save_attackers)

# Display the attackers save button
display(attacker_save_button)

# Read the CSV file for defenders into a pandas dataframe
df2 = pd.read_csv('countries.csv')

# Extract the names of the countries into a list
country_names2 = df2['name'].tolist()

# Create the dropdown widget for defenders
defender_selection = widgets.SelectMultiple(
    options=country_names2,
    value=[],
    description='Defenders:',
    disabled=False
)

# Display the defenders dropdown widget
display(defender_selection)

# Create the button to save defenders
defender_save_button = widgets.Button(
    description='Save Defenders',
    disabled=False,
    button_style='',
    tooltip='Save selected defenders'
)

# Define the function to save defenders
def save_defenders(_):
    with open('defenders.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['name'])
        for defender in defender_selection.value:
            writer.writerow([defender])

# Attach the save function to the defenders button
defender_save_button.on_click(save_defenders)

# Display the defenders save button
display(defender_save_button)


SelectMultiple(description='Attackers:', options=('Afghanistan', 'Åland Islands', 'Albania', 'Algeria', 'Ameri…

Button(description='Save Attackers', style=ButtonStyle(), tooltip='Save selected attackers')

SelectMultiple(description='Defenders:', options=('Afghanistan', 'Åland Islands', 'Albania', 'Algeria', 'Ameri…

Button(description='Save Defenders', style=ButtonStyle(), tooltip='Save selected defenders')

In [ ]:
import pandas as pd

# Load the GLOBAL_FIREPOWER.csv file into a DataFrame
global_firepowers_df = pd.read_csv('/content/GLOBAL_FIREPOWER.CSV')

# Load the attackers.csv file into a DataFrame
attackers_df = pd.read_csv('attackers.csv')

# Merge the two DataFrames on the 'name' column
merged_df = pd.merge(attackers_df, global_firepowers_df, left_on='name', right_on='Country', how='left')

# Select only the columns we want in the output file
output_cols = ['Country', 'ISO3', 'Rank', 'Total Population', 'Manpower Available', 'Fit-for-Service',
               'Reaching Military Age', 'Total Military Personnel', 'Active Personnel', 'Reserve Personnel',
               'Total Aircraft Strength', 'Fighter Aircraft', 'Attack Aircraft', 'Transport Aircraft',
               'Trainer Aircraft', 'Total Helicopter Strength', 'Attack Helicopters', 'Combat Tanks',
               'Armored Fighting Vehicles', 'Self-Propelled Artillery', 'Towed Artillery', 'Rocket Projectors',
               'Total Naval Assets', 'Aircraft Carriers', 'Frigates', 'Destroyers', 'Corvettes', 'Submarines',
               'Patrol Craft', 'Mine Warfare Vessels', 'Production (bbl/dy)', 'Consumption (bbl/dy)',
               'Proven Reserves (bbl)', 'Labor Force', 'Merchant Marine Strength', 'Major Ports / Terminals',
               'Roadway Coverage (km)', 'Railway Coverage (km)', 'Serivecable Airports', 'Defense Budget',
               'External Debt', 'Foreign Exchange / Gold', 'Purchasing Power Parity', 'Square Land Area (km)',
               'Coastline (km)', 'Shared Borders (km)', 'Waterways (km)']

# Drop rows with missing data
merged_df.to_csv('attackers_full.csv', index=False)
cleaned_df = pd.read_csv('attackers_full.csv').dropna(how='all')
cleaned_df.to_csv('attackers_full.csv', index=False)

# Save the merged DataFrame as attackers_full.csv
merged_df[output_cols].to_csv('attackers_full.csv', index=False)


In [ ]:
import pandas as pd

# Load the GLOBAL_FIREPOWER.csv file into a DataFrame
global_firepowers_df = pd.read_csv('/content/GLOBAL_FIREPOWER.CSV')

# Load the defenders.csv file into a DataFrame
defenders_df = pd.read_csv('defenders.csv')

# Merge the two DataFrames on the 'name' column
merged_df = pd.merge(defenders_df, global_firepowers_df, left_on='name', right_on='Country', how='left')

# Select only the columns we want in the output file
output_cols = ['Country', 'ISO3', 'Rank', 'Total Population', 'Manpower Available', 'Fit-for-Service',
               'Reaching Military Age', 'Total Military Personnel', 'Active Personnel', 'Reserve Personnel',
               'Total Aircraft Strength', 'Fighter Aircraft', 'Attack Aircraft', 'Transport Aircraft',
               'Trainer Aircraft', 'Total Helicopter Strength', 'Attack Helicopters', 'Combat Tanks',
               'Armored Fighting Vehicles', 'Self-Propelled Artillery', 'Towed Artillery', 'Rocket Projectors',
               'Total Naval Assets', 'Aircraft Carriers', 'Frigates', 'Destroyers', 'Corvettes', 'Submarines',
               'Patrol Craft', 'Mine Warfare Vessels', 'Production (bbl/dy)', 'Consumption (bbl/dy)',
               'Proven Reserves (bbl)', 'Labor Force', 'Merchant Marine Strength', 'Major Ports / Terminals',
               'Roadway Coverage (km)', 'Railway Coverage (km)', 'Serivecable Airports', 'Defense Budget',
               'External Debt', 'Foreign Exchange / Gold', 'Purchasing Power Parity', 'Square Land Area (km)',
               'Coastline (km)', 'Shared Borders (km)', 'Waterways (km)']

# Save the merged DataFrame as defenders_full.csv
merged_df[output_cols].to_csv('defenders_full.csv', index=False)


In [ ]:
import csv

# open the attackers file and read the military personnel count
with open('attackers_full.csv', 'r') as attackers_file:
    attackers_reader = csv.reader(attackers_file)
    next(attackers_reader) # skip header row
    attackers_row = next(attackers_reader)
    attackers_mil_personnel = int(attackers_row[2]) if attackers_row[2] != '' else 0

# open the defenders file and read the military personnel count
with open('defenders_full.csv', 'r') as defenders_file:
    defenders_reader = csv.reader(defenders_file)
    next(defenders_reader) # skip header row
    defenders_row = next(defenders_reader)
    defenders_mil_personnel = int(defenders_row[2]) if defenders_row[2] != '' else 0

# compare the military personnel counts and print the result
if attackers_mil_personnel > defenders_mil_personnel:
    print("Attackers win!")
else:
    print("Defenders win!")


Attackers win!


In [ ]:
import requests

url = 'https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson'
response = requests.get(url)
with open('world-countries.json', 'w') as f:
    f.write(response.text)


In [ ]:
import pandas as pd
import folium
import pycountry

# Load the data
attackers = pd.read_csv('attackers_full.csv')
defenders = pd.read_csv('defenders_full.csv')

# Get the country codes for the attackers and defenders
attackers_countries = attackers['Country'].unique()
defenders_countries = defenders['Country'].unique()

# Initialize a map centered at (0, 0)
m = folium.Map(location=[0, 0], zoom_start=2)

# Add the countries to the map
for country in pycountry.countries:
    if country.alpha_2 in attackers_countries:
        folium.GeoJson(
            data=open('world-countries.json', 'r', encoding='utf-8-sig').read(),
            style_function=lambda x: {'fillColor': 'green', 'color': 'black', 'weight': 1},
            name=country.alpha_3
        ).add_to(m)
    elif country.alpha_2 in defenders_countries:
        folium.GeoJson(
            data=open('world-countries.json', 'r', encoding='utf-8-sig').read(),
            style_function=lambda x: {'fillColor': 'red', 'color': 'black', 'weight': 1},
            name=country.alpha_3
        ).add_to(m)

# Add layer control to the map
folium.LayerControl().add_to(m)

# Display the map
m


In [ ]:
import pandas as pd
import folium
import pycountry

# Load the data
attackers = pd.read_csv('attackers_full.csv')
defenders = pd.read_csv('defenders_full.csv')

# Map country names in CSV files to country names in pycountry
country_mapping = {
    'Bolivia, Plurinational State of': 'Bolivia, Bolivia (Plurinational State of)',
    'Brunei Darussalam': 'Brunei Darussalam',
    'Congo, the Democratic Republic of the': 'Congo, Democratic Republic of the',
    'Czech Republic': 'Czechia',
    'Iran, Islamic Republic of': 'Iran, Islamic Republic of',
    'Korea, Democratic People\'s Republic of': 'Korea, Democratic People\'s Republic of (North Korea)',
    'Korea, Republic of': 'Korea, Republic of (South Korea)',
    'Lao People\'s Democratic Republic': 'Lao People\'s Democratic Republic',
    'Macedonia, the former Yugoslav Republic of': 'North Macedonia',
    'Moldova, Republic of': 'Moldova, Republic of',
    'Russian Federation': 'Russian Federation',
    'Syrian Arab Republic': 'Syrian Arab Republic',
    'Taiwan, Province of China': 'Taiwan, Province of China',
    'Tanzania, United Republic of': 'Tanzania, United Republic of',
    'United States': 'United States of America',
    'Venezuela, Bolivarian Republic of': 'Venezuela, Bolivarian Republic of',
    'Viet Nam': 'Viet Nam',
}

attackers['Country'] = attackers['Country'].map(country_mapping)
defenders['Country'] = defenders['Country'].map(country_mapping)

# Get the country names for the attackers and defenders
attackers_countries = attackers['Country'].unique()
defenders_countries = defenders['Country'].unique()

# Initialize a map centered at (0, 0)
m = folium.Map(location=[0, 0], zoom_start=2)

# Add the countries to the map
for country in pycountry.countries:
    if country.name in attackers_countries:
        folium.GeoJson(
            data=open('world-countries.json', 'r', encoding='utf-8-sig').read(),
            style_function=lambda x: {'fillColor': 'green', 'color': 'black', 'weight': 1},
            name=country.alpha_3
        ).add_to(m)
    elif country.name in defenders_countries:
        folium.GeoJson(
            data=open('world-countries.json', 'r', encoding='utf-8-sig').read(),
            style_function=lambda x: {'fillColor': 'red', 'color': 'black', 'weight': 1},
            name=country.alpha_3
        ).add_to(m)

# Add layer control to the map
folium.LayerControl().add_to(m)

# Display the map
m


In [1]:
import pandas as pd
import folium
import pycountry

# Load the data
attackers = pd.read_csv('attackers_full.csv')
defenders = pd.read_csv('defenders_full.csv')

# Get the ISO3 country codes for the attackers and defenders
attackers_countries = [pycountry.countries.get(name=c).alpha_3 for c in attackers['Country'].unique()]
defenders_countries = [pycountry.countries.get(name=c).alpha_3 for c in defenders['Country'].unique()]

# Initialize a map centered at (0, 0)
m = folium.Map(location=[0, 0], zoom_start=2)

# Add the countries to the map
for country in pycountry.countries:
    if country.alpha_3 in attackers_countries:
        folium.GeoJson(
            data=open('world-countries.json', 'r', encoding='utf-8-sig').read(),
            style_function=lambda x: {'fillColor': 'green', 'color': 'black', 'weight': 1},
            name=country.alpha_3
        ).add_to(m)
    elif country.alpha_3 in defenders_countries:
        folium.GeoJson(
            data=open('world-countries.json', 'r', encoding='utf-8-sig').read(),
            style_function=lambda x: {'fillColor': 'red', 'color': 'black', 'weight': 1},
            name=country.alpha_3
        ).add_to(m)

# Add layer control to the map
folium.LayerControl().add_to(m)

# Display the map
m


ModuleNotFoundError: ignored